In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import timeit
import time
import string


client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Get the Track ID Data

In [44]:
# timeit library to measure the time needed to run this code
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
alphabet_string = string.ascii_uppercase
alphabet_list = list(alphabet_string)
release_date = []

for alpha in alphabet_list:
    for year in range(2010, 2021, 1):
        for i in range(0,1000,50):
            track_results = sp.search(q='track:{} year={}'.format(alpha, year), type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                popularity.append(t['popularity'])
                release_date.append(t['album']['release_date'])

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 926.3114455829996


In [43]:
tmp[0]['album']['release_date']

'2011-05-03'

In [45]:
# for i in range(0,1000,50):
#     track_results = sp.search(q='year:2020', type='track', limit=50,offset=i)
#     for i, t in enumerate(track_results['tracks']['items']):
#         artist_name.append(t['artists'][0]['name'])
#         track_name.append(t['name'])
#         track_id.append(t['id'])
#         popularity.append(t['popularity'])

In [46]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity,'release_date':release_date})
print(df_tracks.shape)
df_tracks.head()

(135395, 5)


,artist_name,track_name,track_id,popularity,release_date
0,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03
1,Colour Girl,Joyrider - Y Tribe's Underground Remix,01GLLCecGsPfuPmFHDqMCu,39,2016-01-29
2,Proper.,2010 Was a Bad Year (Albert's Song),5hLTnrrsYuW0S3Zzm4iNbQ,5,2017-09-28
3,เคโอติก,เหงาปาก ( Alone ),1F0sXGOtHI1vO3rjJvmCnv,32,2010-01-28
4,Traditional,City Called Heaven (arr. J. Poelinitz),6GUR4qdePjsCET9wde2hmp,7,2013-01-29


In [50]:
print(len(track_id))

135395


In [48]:
tmp = set(track_id)
len(tmp)

45537

# Get the Audio Features Data

In [51]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 194
Time to run this code (in seconds): 252.00845441599995


In [52]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (135201, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.295,0.188,11,-14.399,0,0.0381,0.8970,0.000495,0.6980,0.0510,132.638,audio_features,7hrxbKc7sUWng9lTdr6OOz,spotify:track:7hrxbKc7sUWng9lTdr6OOz,https://api.spotify.com/v1/tracks/7hrxbKc7sUWn...,https://api.spotify.com/v1/audio-analysis/7hrx...,291640,3
1,0.878,0.648,6,-6.822,1,0.1760,0.0475,0.046800,0.0360,0.9600,130.838,audio_features,01GLLCecGsPfuPmFHDqMCu,spotify:track:01GLLCecGsPfuPmFHDqMCu,https://api.spotify.com/v1/tracks/01GLLCecGsPf...,https://api.spotify.com/v1/audio-analysis/01GL...,239174,4
2,0.479,0.404,10,-9.723,1,0.0313,0.3900,0.068700,0.1220,0.0378,109.560,audio_features,5hLTnrrsYuW0S3Zzm4iNbQ,spotify:track:5hLTnrrsYuW0S3Zzm4iNbQ,https://api.spotify.com/v1/tracks/5hLTnrrsYuW0...,https://api.spotify.com/v1/audio-analysis/5hLT...,223343,4
3,0.829,0.549,6,-7.844,0,0.0642,0.1600,0.000000,0.0904,0.7900,84.977,audio_features,1F0sXGOtHI1vO3rjJvmCnv,spotify:track:1F0sXGOtHI1vO3rjJvmCnv,https://api.spotify.com/v1/tracks/1F0sXGOtHI1v...,https://api.spotify.com/v1/audio-analysis/1F0s...,220147,4
4,0.245,0.106,10,-21.515,0,0.0655,0.7950,0.000000,0.5780,0.2150,83.371,audio_features,6GUR4qdePjsCET9wde2hmp,spotify:track:6GUR4qdePjsCET9wde2hmp,https://api.spotify.com/v1/tracks/6GUR4qdePjsC...,https://api.spotify.com/v1/audio-analysis/6GUR...,283747,4


In [53]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135201 entries, 0 to 135200
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      135201 non-null  float64
 1   energy            135201 non-null  float64
 2   key               135201 non-null  int64  
 3   loudness          135201 non-null  float64
 4   mode              135201 non-null  int64  
 5   speechiness       135201 non-null  float64
 6   acousticness      135201 non-null  float64
 7   instrumentalness  135201 non-null  float64
 8   liveness          135201 non-null  float64
 9   valence           135201 non-null  float64
 10  tempo             135201 non-null  float64
 11  type              135201 non-null  object 
 12  id                135201 non-null  object 
 13  uri               135201 non-null  object 
 14  track_href        135201 non-null  object 
 15  analysis_url      135201 non-null  object 
 16  duration_ms       13

In [54]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(135201, 14)

In [55]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (135201, 14)


,artist_name,track_name,track_id,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.897,0.000495,0.698,0.051,132.638,291640,3
1,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.897,0.000495,0.698,0.051,132.638,291640,3
2,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.897,0.000495,0.698,0.051,132.638,291640,3
3,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.897,0.000495,0.698,0.051,132.638,291640,3
4,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.897,0.000495,0.698,0.051,132.638,291640,3


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601141 entries, 0 to 601140
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist_name       601141 non-null  object 
 1   track_name        601141 non-null  object 
 2   track_id          601141 non-null  object 
 3   popularity        601141 non-null  int64  
 4   release_date      601141 non-null  object 
 5   danceability      601141 non-null  float64
 6   energy            601141 non-null  float64
 7   key               601141 non-null  int64  
 8   loudness          601141 non-null  float64
 9   mode              601141 non-null  int64  
 10  speechiness       601141 non-null  float64
 11  acousticness      601141 non-null  float64
 12  instrumentalness  601141 non-null  float64
 13  liveness          601141 non-null  float64
 14  valence           601141 non-null  float64
 15  tempo             601141 non-null  float64
 16  duration_ms       60

In [57]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]


,artist_name,track_name,track_id,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.8970,0.000495,0.6980,0.0510,132.638,291640,3
1,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.8970,0.000495,0.6980,0.0510,132.638,291640,3
2,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.8970,0.000495,0.6980,0.0510,132.638,291640,3
3,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.8970,0.000495,0.6980,0.0510,132.638,291640,3
4,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.8970,0.000495,0.6980,0.0510,132.638,291640,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601111,accel,Zombie,541bwdNIp0xAHE5Qs5HpOA,0,2020-04-15,0.438,0.944,7,-2.886,1,0.0997,0.0134,0.000000,0.2550,0.5610,172.148,251813,4
601112,accel,Zombie,541bwdNIp0xAHE5Qs5HpOA,0,2020-04-15,0.438,0.944,7,-2.886,1,0.0997,0.0134,0.000000,0.2550,0.5610,172.148,251813,4
601113,accel,Zombie,541bwdNIp0xAHE5Qs5HpOA,0,2020-04-15,0.438,0.944,7,-2.886,1,0.0997,0.0134,0.000000,0.2550,0.5610,172.148,251813,4
601123,Ran-D,Zombie,5RWQ9gbCEmzeaTMtX1rmcl,21,2017-11-24,0.436,0.869,4,-2.473,0,0.0615,0.0542,0.000000,0.0979,0.0383,155.051,289549,4


In [58]:
test = df.copy()

In [59]:
test = test.drop_duplicates()

In [60]:
test

,artist_name,track_name,track_id,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Brandi Carlile,"Sixty Years On - Live at Benaroya Hall, Seattl...",7hrxbKc7sUWng9lTdr6OOz,22,2011-05-03,0.295,0.188,11,-14.399,0,0.0381,0.897000,0.000495,0.6980,0.0510,132.638,291640,3
64,Colour Girl,Joyrider - Y Tribe's Underground Remix,01GLLCecGsPfuPmFHDqMCu,39,2016-01-29,0.878,0.648,6,-6.822,1,0.1760,0.047500,0.046800,0.0360,0.9600,130.838,239174,4
113,Proper.,2010 Was a Bad Year (Albert's Song),5hLTnrrsYuW0S3Zzm4iNbQ,5,2017-09-28,0.479,0.404,10,-9.723,1,0.0313,0.390000,0.068700,0.1220,0.0378,109.560,223343,4
129,เคโอติก,เหงาปาก ( Alone ),1F0sXGOtHI1vO3rjJvmCnv,32,2010-01-28,0.829,0.549,6,-7.844,0,0.0642,0.160000,0.000000,0.0904,0.7900,84.977,220147,4
130,Traditional,City Called Heaven (arr. J. Poelinitz),6GUR4qdePjsCET9wde2hmp,7,2013-01-29,0.245,0.106,10,-21.515,0,0.0655,0.795000,0.000000,0.5780,0.2150,83.371,283747,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601136,Marco Bocatto,Zinomas,6bJjXslmoACI0ExZUhl7Ef,0,2020-01-09,0.810,0.716,1,-10.824,1,0.1020,0.001140,0.846000,0.5500,0.6950,121.972,466099,4
601137,Xenias,Zippers,5HiJErsYWyT8a0qiSUnEL2,0,2020-01-11,0.701,0.502,1,-10.629,0,0.0862,0.000042,0.908000,0.6720,0.5780,128.013,207187,4
601138,Alex Gor,Zestful Archangel,3SOO4nx2FukpTcBc1lmXer,0,2020-02-29,0.691,0.801,9,-7.297,0,0.0449,0.282000,0.929000,0.1360,0.4910,130.019,243692,4
601139,Elda Sartori,Zen Space,4ZrJ3avTDjHrv7bklAW80J,0,2020-01-09,0.175,0.129,3,-23.052,1,0.0352,0.857000,0.944000,0.1350,0.0360,63.772,309000,5


In [61]:
test.to_csv('dataset.csv')